# Problem Statement

## Business Context

In the highly competitive telecommunications industry, customer retention is critical to sustaining growth and profitability. Customer churn remains a persistent challenge, making it essential to understand customer behavior and the underlying factors that drive their decision to leave. Gaining these insights is key to maintaining loyalty and delivering superior service.

To tackle this, the Customer Analytics & Retention Department has been analyzing historical customer data, segmented into two main groups—customers who have churned and those who have been retained. By leveraging advanced machine learning techniques, the team has developed a predictive model to identify patterns associated with churn risk. The goal is to provide actionable insights that enable proactive, targeted retention strategies.

However, several challenges hinder the effectiveness of the current process:

1. **Data Overload**: The vast volume and complexity of customer data make it difficult to extract timely, meaningful insights, slowing down decision-making and intervention.
2. **Delayed Responses**: The existing churn analysis approach lacks the speed needed to respond to emerging trends, often resulting in missed retention opportunities.
3. **Limited Accessibility**: For the model to be impactful, it must be easily accessible to all customer-facing teams. A user-friendly web application is essential to ensure its widespread adoption and practical use.


## Objective

To ensure seamless and scalable access to the churn prediction system, the Customer Analytics & Retention Department seeks to improve how the model and its environment are deployed across teams. The current centralized web-based deployment has caused performance issues and high latency as access expanded to distributed locations. Additionally, sharing the model has been problematic due to environment and system incompatibilities.

The department's objective is to develop a standardized, portable solution that packages the model, dependencies, configurations, and runtime environment into a self-contained unit. This approach aims to eliminate compatibility issues, reduce errors during deployment, simplify setup, and enable reliable use of the application across different systems. The goal is to empower all teams with consistent, low-latency access to the churn prediction model, supporting timely and proactive customer retention efforts.

# App Backend

## Points to note before executing the below cells
- Visit [this](https://huggingface.co/new-space) link to create a new space
  - Under the space creation, enter the below details
    - Space name: **Backend**
(If you were trying with different names, be cautious when using a underscore `_` in space names, such as `backend_space`, as it can cause exceptions when accessing the API URL. Always use hyphen `-` instead, like `backend-space`.)
    - Select the space SDK: **Docker**
    - Choose a Docker tempplate: **Blank**
    - Click on **Create Space**

## Flask Web framework

Flask is a lightweight, flexible Python web framework used to build web applications and APIs quickly and easily.

Flask allows you to:
- Create web routes (URLs that users can access)
- Handle HTTP requests and responses
- Build REST APIs to expose machine learning models or business logic
- Serve HTML templates, form inputs, and JSON data

This API allows any application (like a dashboard, CRM, or mobile app) to send customer data and get back a prediction on whether the customer is likely to churn.

In [ ]:
# Create a folder for storing the files needed for backend server deployment
import os
os.makedirs("backend_files", exist_ok=True)

In [ ]:
%%writefile backend_files/app.py
import joblib
import pandas as pd
from flask import Flask, request, jsonify

# Initialize Flask app with a name
app = Flask("Telecom Customer Churn Predictor")

# Load the trained churn prediction model
model = joblib.load("churn_prediction_model_v1_0.joblib")

# Define a route for the home page
@app.get('/')
def home():
    return "Welcome to the Telecom Customer Churn Prediction API"

# Define an endpoint to predict churn for a single customer
@app.post('/v1/customer')
def predict_churn():
    # Get JSON data from the request
    customer_data = request.get_json()

    # Extract relevant customer features from the input data
    sample = {
        'SeniorCitizen': customer_data['SeniorCitizen'],
        'Partner': customer_data['Partner'],
        'Dependents': customer_data['Dependents'],
        'tenure': customer_data['tenure'],
        'PhoneService': customer_data['PhoneService'],
        'InternetService': customer_data['InternetService'],
        'Contract': customer_data['Contract'],
        'PaymentMethod': customer_data['PaymentMethod'],
        'MonthlyCharges': customer_data['MonthlyCharges'],
        'TotalCharges': customer_data['TotalCharges']
    }

    # Convert the extracted data into a DataFrame
    input_data = pd.DataFrame([sample])

    # Make a churn prediction using the trained model
    prediction = model.predict(input_data).tolist()[0]

    # Map prediction result to a human-readable label
    prediction_label = "churn" if prediction == 1 else "not churn"

    # Return the prediction as a JSON response
    return jsonify({'Prediction': prediction_label})

# Define an endpoint to predict churn for a batch of customers
@app.post('/v1/customerbatch')
def predict_churn_batch():
    # Get the uploaded CSV file from the request
    file = request.files['file']

    # Read the file into a DataFrame
    input_data = pd.read_csv(file)

    # Make predictions for the batch data and convert raw predictions into a readable format
    predictions = [
        'Churn' if x == 1
        else "Not Churn"
        for x in model.predict(input_data.drop("customerID",axis=1)).tolist()
    ]

    cust_id_list = input_data.customerID.values.tolist()
    output_dict = dict(zip(cust_id_list, predictions))

    return output_dict

# Run the Flask app in debug mode
if __name__ == '__main__':
    app.run(debug=True)

Writing backend_files/app.py


In the given code snippet, the decorators `@app.get('/')`, `@app.post('/v1/customer')`, and `@app.post('/v1/customerbatch')` are used in **FlaskAPI**, a modern web framework for building APIs in Python.

**What is a Decorator?**\
In Python, a **decorator** is a function that modifies the behavior of another function. The `@` syntax is used to apply a decorator.


**`@app.get('/')`**
- Defines a **GET endpoint** at the root URL `/`.
- When someone sends a GET request to `/`, the `home()` function will be triggered.
- Returns a welcome message.
- Whenever a user clicks on the deployment web app link or API link, a GET request is automatically triggered in the background and sent to the API. As defined in the function, this request targets the homepage, and the user is greeted with a welcome message.

**`@app.post('/v1/customer')`**
- Defines a **POST endpoint** at `/v1/customer`.
- Typically used to send customer data (like JSON payload) for **churn prediction** for a **single customer**.
- Which return the json object, as we have given "return" statement in the predict_churn function

**`@app.post('/v1/customerbatch')`**
- Defines another **POST endpoint**, but for **batch predictions**.
- Sending multiple customer data points at once (e.g., a list of JSON objects) and returns churn predictions for each, as we have given "return" statement in the predict_churn_batch function


**Summary of flow of script**
1. **Model Loading (`joblib.load`)**  
   Loads a pre-trained ML model (`churn_prediction_model_v1_0.joblib`) trained to predict churn based on customer features.

2. **Flask App Setup**  
   Initializes a simple Flask web app (`Flask("Telecom Customer Churn Predictor")`), which exposes endpoints over HTTP.

3. **Root Route (`/`)**  
   Returns a welcome message for basic API connectivity testing.

4. **Single Customer Prediction (`/v1/customer`)**
   - Accepts a **JSON payload** with a single customer's features.
   - Converts the data into a Pandas DataFrame.
   - Runs the model’s `.predict()` method.
   - Returns a **human-readable result**: `"churn"` or `"not churn"`.

5. **Batch Customer Prediction (`/v1/customerbatch`)**
   - Accepts a **CSV file upload** via form-data.
   - Reads customer records into a DataFrame.
   - Predicts churn for **multiple customers at once**.
   - Returns results in a dictionary: `{customer_id: prediction}` format.

6. **Model Inference**
   - Makes use of `.predict()` to infer whether customers will churn.

7. **Debug Mode (`app.run(debug=True)`)**
   - Allows hot-reloading, i.e., restart whenever you make changes to your Python code by reloading the updated version of your app without needing to manually stop and restart the server
   - Enables useful logs during deployment

**Why This Is Valuable**
- **Real-time ML integration:** Bridges the gap between data science and real-world business applications.
- **Scalable predictions:** Enables both real-time (single) and batch processing of customers.
- **Automation-ready:** Can trigger automated actions (e.g., customer retention emails) based on predictions.


## Dependencies file

In [ ]:
%%writefile backend_files/requirements.txt
pandas==2.2.2
numpy==2.0.2
scikit-learn==1.6.1
xgboost==2.1.4
joblib==1.4.2
Werkzeug==2.2.2
flask==2.2.2
gunicorn==20.1.0
requests==2.28.1
uvicorn[standard]

Writing backend_files/requirements.txt


1. **`flask==2.2.2`**
- Flask is a lightweight and flexible web framework for Python that allows developers to build web applications quickly and easily. It is designed with simplicity in mind, providing the essentials to get a web app up and running without unnecessary complexity. Flask supports extensions for added functionality and is known for its ease of use, making it a popular choice for both beginners and experienced developers. For more information, click on the link [here](https://flask.palletsprojects.com/en/stable/)

---

2. **`Werkzeug==2.2.2`**
- Werkzeug is a comprehensive WSGI (Web Server Gateway Interface) utility library for Python, designed to help build web applications in a flexible and modular way. It provides tools for request and response handling, routing, and session management, among other features. Often used as a foundation for popular web frameworks like Flask, Werkzeug allows developers to create efficient and reliable web services. For more information, click on the link [here](https://werkzeug.palletsprojects.com/en/stable/)

---

3. **`gunicorn==20.1.0`**
- Gunicorn (Green Unicorn) is a web server for Python that helps run web applications. It can handle multiple requests at the same time by using several worker processes, which makes it faster and more efficient. Gunicorn is easy to set up and is often used with popular Python web frameworks like Flask and Django to make sure websites run smoothly in production. For more information, click on the link [here](https://docs.gunicorn.org/en/stable/)

---

4. **`requests==2.28.1`**
- Requests is a popular Python library that makes it easy to send HTTP requests and handle responses. It simplifies making GET, POST, PUT, and DELETE requests to web servers and makes it easy to work with APIs. With a simple and intuitive interface, Requests helps developers communicate with web services effortlessly, handling things like URL encoding, authentication, and session management. For more information, click on the link [here](https://requests.readthedocs.io/en/latest/)

---

5. **`uvicorn[standard]`**
- Uvicorn is a lightning-fast ASGI (Asynchronous Server Gateway Interface) web server for Python, designed to run asynchronous web applications. It is compatible with modern frameworks like FastAPI and Starlette, enabling high-performance handling of web requests with support for both HTTP/1.1 and HTTP/2. Uvicorn is easy to set up and is ideal for building scalable web applications that require asynchronous capabilities. For more information, click on the link [here](https://www.uvicorn.org/)

**Summary Table**

| Package         | Role                            | Use Case                        |
|----------------|----------------------------------|----------------------------------|
| **Flask**       | Web framework                   | Build API                        |
| **Werkzeug**    | WSGI engine, routing, utils     | Used internally by Flask         |
| **Gunicorn**    | WSGI server                     | Run Flask in production          |
| **Requests**    | HTTP client                     | Call/test APIs                   |
| **Uvicorn**     | ASGI server                     | Run async applications      |

## Dockerfile

In [ ]:
%%writefile backend_files/Dockerfile
FROM python:3.9-slim

# Set the working directory inside the container
WORKDIR /app

# Copy all files from the current directory to the container's working directory
COPY . .

# Install dependencies from the requirements file without using cache to reduce image size
RUN pip install --no-cache-dir -r requirements.txt

# Define the command to start the application using Gunicorn with 4 worker processes
# - `-w 4`: Uses 4 worker processes for handling requests
# - `-b 0.0.0.0:7860`: Binds the server to port 7860 on all network interfaces
# - `app:app`: Runs the Flask app (assuming `app.py` contains the Flask instance named `app`)
CMD ["gunicorn", "-w", "4", "-b", "0.0.0.0:7860", "app:app"]

Writing backend_files/Dockerfile


This Dockerfile is used to **containerize your Flask application**, making it easy to deploy and run consistently across different environments. Let’s walk through what each line does and why it’s important:

---

**Line-by-Line Explanation**

`FROM python:3.9-slim`
- **What:** Starts from a lightweight version of Python 3.9; is fetched from [Docker Hub](https://hub.docker.com/layers/library/python/3.9-slim/images/sha256-b370e60efdfcc5fcb0a080c0905bbcbeb1060db3ce07c3ea0e830b0d4a17f758)
- **Why:** It keeps the image small and efficient while still having everything needed to run Python apps.

---

`WORKDIR /app`
- **What:** Sets the working directory inside the container to `/app`.
- **Why:** All subsequent commands (copying files, running commands) happen inside this folder, keeping things organized.

---

`COPY . .`
- **What:** Copies everything from your local project directory into the `/app` folder inside the container.
- **Why:** This ensures your source code, models, and configuration files are available in the container.

---

`RUN pip install --no-cache-dir -r requirements.txt`
- **What:** Installs all Python dependencies listed in `requirements.txt`.
- **Why:** These are needed to run your Flask app, and using `--no-cache-dir` keeps the image smaller by not storing temporary installation files.

---

`CMD ["gunicorn", "-w", "4", "-b", "0.0.0.0:7860", "app:app"]`
- **What:** This tells the container what command to run when it starts:
  - `gunicorn` is a production-grade Python WSGI server.
  - `-w 4` means it will use 4 worker processes (good for handling multiple requests).
  - `-b 0.0.0.0:7860` binds the app to all IP addresses on port 7860.
  - `app:app` means it will look for a file called `app.py`, and within it, the Flask instance named `app`.

- **Why:** Gunicorn is preferred over Flask’s development server for production use. It handles multiple users more efficiently and securely.

---

**What This Dockerfile Achieves**
- Packages your Flask app and dependencies into a single container.
- Ensures consistency across environments (e.g., local machine, cloud server, Hugging Face Space).
- Uses Gunicorn for scalable, production-ready deployment.

## Uploading Files to Hugging Face Space for the Backend

**Note**: Before running the code below, ensure that the serialized ML model has been uploaded in to `backend_files` folder.

In [ ]:
# for hugging face space authentication to upload files
from huggingface_hub import login, HfApi

access_key = "---------------------"  # Your Hugging Face token created from access keys in write mode
repo_id = "user_name/space_name"  # Your Hugging Face space id

# Login to Hugging Face platform with the access token
login(token=access_key)

# Initialize the API
api = HfApi()

# Upload Streamlit app files stored in the folder called deployment_files
api.upload_folder(
    folder_path="/content/backend_files",  # Local folder path
    repo_id=repo_id,  # Hugging face space id
    repo_type="space",  # Hugging face repo type "space"
)

# App Frontend

## Points to note before executing the below cells
- Create a Streamlit space on Hugging Face by following the instructions provided on the content page titled **`Creating Spaces and Adding Secrets in Hugging Face`** from Week 1

## Streamlit for Interactive UI

In [ ]:
# Create a folder for storing the files needed for frontend UI deployment
os.makedirs("frontend_files", exist_ok=True)

In [ ]:
%%writefile frontend_files/app.py
import streamlit as st
import pandas as pd
import requests

# Streamlit UI for Customer Churn Prediction
st.title("Telecom Customer Churn Prediction App")
st.write("This tool predicts customer churn risk based on their details. Enter the required information below.")

# Collect user input based on dataset columns
CustomerID = st.number_input("Customer ID", min_value=10000000, max_value=99999999)
SeniorCitizen = st.selectbox("Senior citizen", ["Yes", "No"])
Partner = st.selectbox("Does the customer have a partner?", ["Yes", "No"])
Dependents = st.selectbox("Does the customer have dependents?", ["Yes", "No"])
PhoneService = st.selectbox("Does the customer have phone service?", ["Yes", "No"])
InternetService = st.selectbox("Type of Internet Service", ["DSL", "Fiber optic", "No"])
Contract = st.selectbox("Type of Contract", ["Month-to-month", "One year", "Two year"])
PaymentMethod = st.selectbox("Payment Method", ["Electronic check", "Mailed check", "Bank transfer", "Credit card"])
tenure = st.number_input("Tenure (Months with the company)", min_value=0, value=12)
MonthlyCharges = st.number_input("Monthly Charges", min_value=0.0, value=50.0)
TotalCharges = st.number_input("Total Charges", min_value=0.0, value=600.0)

# Convert categorical inputs to match model training
customer_data = {
    'SeniorCitizen': 1 if SeniorCitizen == "Yes" else 0,
    'Partner':Partner,
    'Dependents': Dependents,
    'tenure': tenure,
    'PhoneService': PhoneService,
    'InternetService': InternetService,
    'Contract': Contract,
    'PaymentMethod': PaymentMethod,
    'MonthlyCharges': MonthlyCharges,
    'TotalCharges': TotalCharges
}


if st.button("Predict", type='primary'):
    response = requests.post("https://<user_name>-<space_name>.hf.space/v1/customer", json=customer_data)    # enter user name and space name before running the cell
    if response.status_code == 200:
        result = response.json()
        churn_prediction = result["Prediction"]  # Extract only the value
        st.write(f"Based on the information provided, the customer with ID {CustomerID} is likely to {churn_prediction}.")
    else:
        st.error("Error in API request")

# Batch Prediction
st.subheader("Batch Prediction")

file = st.file_uploader("Upload CSV file", type=["csv"])
if file is not None:
    if st.button("Predict for Batch", type='primary'):
        response = requests.post("https://<user_name>-<space_name>.hf.space/v1/customerbatch", files={"file": file})    # enter user name and space name before running the cell
        if response.status_code == 200:
            result = response.json()
            st.header("Batch Prediction Results")
            st.write(result)
        else:
            st.error("Error in API request")

Overwriting frontend_files/app.py




**Purpose**
This app allows users (business teams, analysts, etc.) to **interact with your churn prediction model** using a friendly interface - no coding needed.

---

**What It Contains**

1. **User Inputs for a Single Customer**
- Sends a **JSON request** to your `/v1/customer` API using `requests.post()`.

---

**Batch Prediction with CSV Upload**
- Allows uploading a CSV file (e.g., list of customer records).
- Sends it to the `/v1/customerbatch` API endpoint.
- Displays the results in a clean dictionary-style table using `st.write()`.

---

**Why This UI Is Important**

- **Non-technical users** can try the model easily.
- Supports both **ad-hoc testing** and **batch predictions**.
- Can be hosted on **Hugging Face Spaces**, **Streamlit Cloud**, or internal tools.

## Dependencies file

In [ ]:
%%writefile frontend_files/requirements.txt
pandas==2.2.2
requests==2.28.1
streamlit==1.43.2

Overwriting frontend_files/requirements.txt


## Dockerfile

In [ ]:
%%writefile frontend_files/Dockerfile
# Use a minimal base image with Python 3.9 installed
FROM python:3.9-slim

# Set the working directory inside the container to /app
WORKDIR /app

# Copy all files from the current directory on the host to the container's /app directory
COPY . .

# Install Python dependencies listed in requirements.txt
RUN pip3 install -r requirements.txt

# Define the command to run the Streamlit app on port 8501 and make it accessible externally
CMD ["streamlit", "run", "app.py", "--server.port=8501", "--server.address=0.0.0.0", "--server.enableXsrfProtection=false"]

Overwriting frontend_files/Dockerfile


## Uploading Files to Hugging Face Repository

In [ ]:
access_key = "---------------------"  # Your Hugging Face token created from access keys in write mode
repo_id = "user_name/space_name"  # Your Hugging Face space id

# Login to Hugging Face platform with the access token
login(token=access_key)

# Initialize the API
api = HfApi()

# Upload Streamlit app files stored in the folder called deployment_files
api.upload_folder(
    folder_path="/content/frontend_files",  # Local folder path
    repo_id=repo_id,  # Hugging face space id
    repo_type="space",  # Hugging face repo type "space"
)

# Question - Why do we need streamlit app.py when we have a flask app.py to make predictions?

You need both the **Flask app** and the **Streamlit app** because they serve **two different roles** in your project.

The **Flask app** is your **backend**. It holds the trained machine learning model and handles the actual prediction logic. When someone sends customer data to this Flask API, it processes that data and returns a prediction—whether the customer will churn or not. But the Flask app by itself doesn’t offer any user interface; it just waits for data, makes predictions, and sends back a response in code form (like JSON).

The **Streamlit app**, on the other hand, is your **frontend**. It’s built for people to interact with—especially non-technical users. It provides a clean interface with sliders, dropdowns, and file upload buttons. Users can input customer details or upload a CSV file, and the Streamlit app will send that data to the Flask API in the background. When it receives the prediction, it presents the result in a user-friendly way—like showing “This customer is likely to churn.”

So, in short:
- **Flask** does the smart stuff in the background.
- **Streamlit** helps users talk to that smart system without needing to write any code.

---

**Frontend–Backend Workflow**

| Frontend (Streamlit)                 | Backend (Flask API)                |
|-------------------------------------|------------------------------------|
| Collects user input                 | Receives JSON or CSV               |
| Converts input to correct format    | Parses input using `request.get_json()` or `request.files` |
| Sends `POST` request via `requests` | Runs the model + returns result    |
| Displays prediction in human-readable form                 | Returns `"churn"` or `"not churn"` |

# Inferencing using Flask API


Lets see how to interact with a deployed Flask API using programatically on Hugging Face Spaces to perform **online** and **batch inference**  

We will:  
1. Send API requests for both online and batch inference.  
2. Process and visualize the model predictions.  

This ensures seamless interaction with the deployed model while leveraging the API for scalable inference.

In [ ]:
import json  # To handle JSON formatting for API requests and responses
import requests  # To send HTTP requests to the deployed Flask API

import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical computations

from sklearn.model_selection import train_test_split  # To split data for batch inference scenarios

In [ ]:
model_root_url = "https://<user_name>-<space_name>.hf.space/"  # Base URL of the deployed Flask API on Hugging Face Spaces

In [ ]:
model_url = model_root_url + "/v1/customer"  # Endpoint for online (single) inference

Since our model predictions are provided by the following endpoint we created using Flask, we need to invoke the same to make prediction.

> ```@app.post('/v1/customer')```

In [ ]:
model_batch_url = model_root_url + "/v1/customerbatch"  # Endpoint for batch inference

> ```@app.post('/v1/customerbatch')```

## Online

**Online Inference:** Sending a single request to the API and receiving an immediate response. This is useful for real-time applications like recommendation systems and fraud detection.  
* This data is sent as a JSON payload in a POST request to the model endpoint.

* The model processes the input features and returns a prediction.

In [ ]:
payload = {
  'SeniorCitizen': 0,
 'Partner': 'Yes',
 'Dependents': 'No',
 'tenure': 63,
 'PhoneService': 'No',
 'InternetService': 'Fiber optic',
 'Contract': 'One year',
 'PaymentMethod': 'Credit card',
 'MonthlyCharges': 48.66,
 'TotalCharges': 3065.58
}

In [ ]:
# Sending a POST request to the model endpoint with the test payload
response = requests.post(model_url, json=payload)

In [ ]:
response

<Response [200]>

`<Response [200]>` indicates that the HTTP request was successful.  

- `Response` is the object returned by `requests.post()`.  
- `[200]` is the HTTP status code, meaning **OK** (the request was processed successfully).  


In [ ]:
print(response.json())

{'Prediction': 'churn'}


`response.json()` is used to parse the response body as JSON.  
- If the API returns data in JSON format, `.json()` converts it into a Python dictionary.  
- This allows easy access to specific values using keys.  

## Batch

**Batch Inference:** Sending multiple inputs in a single request, allowing efficient processing of large datasets. This is ideal for analyzing historical data at scale.

In [ ]:
  batch_churn_dataset = pd.read_csv("Batch-data-telecom.csv")

In [ ]:
# Prepare batch input for API request
batch_input = {
    'file': batch_churn_dataset.to_csv(header=True, index=False).encode('utf-8')
}

This code prepares the customer churn data as a **file-like object** to send in an API request (in the `files` parameter of `requests.post`).

- `batch_churn_dataset`:  
  This is a Pandas DataFrame containing customer data loaded from a CSV file:

- `.to_csv(header=True, index=False)`:  
  Converts the DataFrame into a CSV **string**, keeping the column headers (`header=True`) but **excluding the index** (`index=False`).

- `.encode('utf-8')`:  
  Encodes the CSV string into **bytes** (UTF-8 format), which is necessary for sending it over an HTTP request.

- `'file': ...`:  
  Wraps the encoded CSV data in a dictionary with the key `'file'`, which is expected by the FlaskAPI backend using `UploadFile`.

In [ ]:
# Send request to the model API for batch predictions
response = requests.post(
    model_batch_url,  # Model endpoint URL
    files=batch_input
)

This line sends a **POST request** to the deployed model API to perform **batch churn prediction**.

- `requests.post(...)`:  
  This uses the `requests` library to make a **POST** request to a web server (in this case, our API endpoint for batch predictions).

- `model_batch_url`:  
  This is the URL endpoint where the batch prediction API is hosted. It should look like this:  
  `"https://<user_name>-<space_name>.hf.space/v1/customerbatch"`

- `files=batch_input`:  
  This sends the batch data file (e.g., a CSV) as part of the request using the `files` parameter.  

- `response`:  
  This is the result of the API call and contains the response from the server, including predicted churn values.

In [ ]:
response

<Response [200]>

In [ ]:
# Extract predictions from API response
response.text

'{"CUST0048":"Not Churn","CUST0049":"Churn","CUST0050":"Not Churn","CUST0051":"Churn","CUST0052":"Churn","CUST0053":"Not Churn","CUST0054":"Churn","CUST0055":"Not Churn","CUST0056":"Churn","CUST0057":"Churn","CUST0058":"Churn"}\n'

As we can see, we receive a JSON where each key represents a customer ID, and the value represents the model prediction of whether the customer will churn or not.

<font size=6 color="blue">Power Ahead!</font>
___